# Wines Points prediction 

In [1]:
%load_ext autoreload
%autoreload 2
import nltk
import sys; sys.path.append('../')
import pandas as pd
!pip install --upgrade scikit-learn
from sklearn import set_config
set_config(display='diagram')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 7.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
wine_reviews = pd.read_csv("wine_reviews.csv")
wine_reviews.head()

,Unnamed: 0,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,description_len,country1,year,pureTextdescription
0,0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87.0,19.0,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24,Unknown,2013,aromas include tropical fruit broom brimston...
1,1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87.0,15.0,Douro,Unknown,Unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,38,Unknown,2011,this is ripe and fruity a wine that is smooth...
2,2,2,US,"Tart and snappy, the flavors of lime flesh and...",Unknown,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28,Unknown,2013,tart and snappy the flavors of lime flesh and...
3,3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,Unknown,Alexander Peartree,Unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33,Unknown,2013,pineapple rind lemon pith and orange blossom ...
4,4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41,Unknown,2012,much like the regular bottling from this come...


In [16]:
from sklearn.model_selection import train_test_split
# Split the data into train and test sets
wine_reviews_X = wine_reviews.drop(columns='points').head(30000)
wine_reviews_y = wine_reviews['points'].head(30000)
X_train, X_test, y_train, y_test = train_test_split(wine_reviews_X, wine_reviews_y, test_size=0.2, random_state=42)

In [17]:
stopwords=nltk.corpus.stopwords.words("english")
stopwords[:5]

['i', 'me', 'my', 'myself', 'we']

In [18]:
nltk.word_tokenize(wine_reviews_X.pureTextdescription.iloc[0])

['aromas',
 'include',
 'tropical',
 'fruit',
 'broom',
 'brimstone',
 'and',
 'dried',
 'herb',
 'the',
 'palate',
 'isnt',
 'overly',
 'expressive',
 'offering',
 'unripened',
 'apple',
 'citrus',
 'and',
 'dried',
 'sage',
 'alongside',
 'brisk',
 'acidity']

In [19]:
train_words = [word for description in wine_reviews.pureTextdescription for word in nltk.word_tokenize(description) if word.lower() not in stopwords] # Words without stop words
# ### final words set
words = pd.DataFrame(data = train_words, columns = ['word']).word.value_counts().reset_index()
words.columns = ['word','wordCount']
words_df = words[(words['wordCount']>20)&(words['wordCount']<len(X_train)/2)]
words_df

,word,wordCount
32,shows,11511
33,light,11436
34,citrus,11366
35,dark,11325
36,bright,10963
...,...,...
6310,shapely,21
6311,uncrushed,21
6312,damask,21
6313,rubyred,21


In [20]:
clean_vocab = set(words_df.word)
len(clean_vocab)

6283

In [21]:
clean_vocab_loop = clean_vocab.copy()
for word,wordCount in zip(words_df.word,words_df.wordCount):
    if wordCount<3:
        # print(word)
        clean_vocab.remove(word)

In [22]:
len(clean_vocab)

6283

In [23]:
wine_reviews.shape

(129908, 19)

In [24]:
results_df = pd.DataFrame(columns = ['method', 'variables', 'parameters', 'train_MSE', 'test_MSE'])

In [25]:
import pandas as pd
import cufflinks as cf;
# cf.go_offline()

In [26]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [27]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn import set_config

ImportError: cannot import name '_fit_context' from 'sklearn.base' (/Users/sachar/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py)

In [ ]:
categorical_features =   ['country', 'province', 'region_1', 'variety', 'winery', 'year']
numerical_features = ['price']
description_features = ['pureTextdescription']

description_transformer = Pipeline(steps=[
    ("squeez", FunctionTransformer(lambda x: x.squeeze())),
    #("vect", CountVectorizer(vocabulary = clean_vocab, tokenizer=nltk.word_tokenize)),
    ("tfidf", TfidfVectorizer(vocabulary=clean_vocab)),
    ("toarray", FunctionTransformer(lambda x: x.toarray())),
])

x_tranformer = ColumnTransformer(
    transformers=[
        ('numerical', MinMaxScaler(), numerical_features),
        ('categorical', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_features),
        ('descriptional', description_transformer, description_features)
], remainder='drop')

x_tranformer.fit_transform(X_train).shape


## KNN

In [ ]:
KNN_pipe = Pipeline([
    ('tranformer', x_tranformer),
    ('classifier', KNeighborsRegressor(n_jobs = -1))
])

In [ ]:
KNN_pipe

In [ ]:
param_grid = {
    'classifier__n_neighbors': [3, 12, 35],
    'classifier__weights': ['uniform', 'distance']
}
# scoring = 'r2'
scoring = 'neg_mean_squared_error'
KNN_grid_search = GridSearchCV(KNN_pipe,param_grid, cv = 5, scoring = scoring, verbose = 3)
KNN_grid_search

In [ ]:
KNN_grid_search.fit(X_train ,y_train)

In [ ]:
KNN_results_df = pd.DataFrame(KNN_grid_search.cv_results_)
KNN_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]

In [85]:
KNN_results_df = pd.DataFrame(KNN_grid_search.cv_results_)
KNN_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
params = KNN_results_df.loc[KNN_results_df.rank_test_score == 1, 'params'].values[0]
train_MSE = -KNN_results_df.loc[KNN_results_df.rank_test_score == 1, 'mean_test_score'].values[0]
test_MSE = mean_squared_error(y_test.values, KNN_grid_search.predict(X_test))
results_df.loc[0] = (['KNN', numerical_features + categorical_features, params, train_MSE, test_MSE])
results_df

,method,variables,parameters,train_MSE,test_MSE
0,KNN,"[price, country, province, region_1, variety, ...","{'classifier__n_neighbors': 12, 'classifier__w...",5.829481,5.812558


## LinearRegression

In [86]:
# scoring = 'neg_mean_squared_error'
scoring = 'r2'

In [87]:
param_grid = [{ 
        'classifier': [LinearRegression()],
        'classifier__fit_intercept': [True, False]
    }]

In [88]:
LinearRegression_pipe = Pipeline([
    ('tranformer', x_tranformer),
    ('classifier', LinearRegression(n_jobs = -1))
])

LinearRegression_grid_search = GridSearchCV(LinearRegression_pipe,param_grid, cv = 5, scoring = scoring, verbose = 3)

In [89]:
LinearRegression_grid_search.fit(X_train ,y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END classifier=LinearRegression(), classifier__fit_intercept=True;, score=0.250 total time= 1.6min
[CV 2/5] END classifier=LinearRegression(), classifier__fit_intercept=True;, score=0.261 total time= 1.5min
[CV 3/5] END classifier=LinearRegression(), classifier__fit_intercept=True;, score=0.303 total time= 1.5min
[CV 4/5] END classifier=LinearRegression(), classifier__fit_intercept=True;, score=0.299 total time= 1.5min
[CV 5/5] END classifier=LinearRegression(), classifier__fit_intercept=True;, score=0.274 total time= 1.5min
[CV 1/5] END classifier=LinearRegression(), classifier__fit_intercept=False;, score=-0.176 total time= 1.5min
[CV 2/5] END classifier=LinearRegression(), classifier__fit_intercept=False;, score=-0.195 total time= 1.5min
[CV 3/5] END classifier=LinearRegression(), classifier__fit_intercept=False;, score=-0.044 total time=65.8min
[CV 4/5] END classifier=LinearRegression(), classifier__fit_intercept=

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tranformer',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         MinMaxScaler(),
                                                                         ['price']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['country',
                                                                          'province',
                                                                          'region_1',
                                                                          'variety',
                                                                          'winery',
                                                                          'year']),
                                                                        ('descriptional',
                                                                         Pipeline(steps=[('squeez',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c...
                                                                                                                      'accentuating',
                                                                                                                      'acceptable',
                                                                                                                      'accessible',
                                                                                                                      'accompanied',
                                                                                                                      'accompanies',
                                                                                                                      'accompaniment', ...})),
                                                                                         ('toarray',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c5551af70>))]),
                                                                         ['pureTextdescription'])])),
                                       ('classifier',
                                        LinearRegression(n_jobs=-1))]),
             param_grid=[{'classifier': [LinearRegression()],
                          'classifier__fit_intercept': [True, False]}],
             scoring='r2', verbose=3)

In [90]:
LinearRegression_results_df = pd.DataFrame(LinearRegression_grid_search.cv_results_)
LinearRegression_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
params = LinearRegression_results_df.loc[LinearRegression_results_df.rank_test_score == 1, 'params'].values[0]
train_MSE = -LinearRegression_results_df.loc[LinearRegression_results_df.rank_test_score == 1, 'mean_test_score'].values[0]
test_MSE = mean_squared_error(y_test.values, LinearRegression_grid_search.predict(X_test))
results_df.loc[1] = (['LinearRegression', numerical_features + categorical_features, params, train_MSE, test_MSE])
results_df

,method,variables,parameters,train_MSE,test_MSE
0,KNN,"[price, country, province, region_1, variety, ...","{'classifier__n_neighbors': 12, 'classifier__w...",5.829481,5.812558
1,LinearRegression,"[price, country, province, region_1, variety, ...","{'classifier': LinearRegression(), 'classifier...",-0.277362,15.264439


## Lasso

In [91]:
# scoring = 'neg_mean_squared_error'
scoring = 'r2'

In [92]:
param_grid = [{ 
        'classifier':[Lasso()],
        'classifier__alpha':[0.01,0.1, 1.0, 10, 100]
    }]

In [93]:
Lasso_pipe = Pipeline([
    ('tranformer', x_tranformer),
    ('classifier', Lasso())
])

Lasso_grid_search = GridSearchCV(Lasso_pipe,param_grid, cv = 5, scoring = scoring, verbose = 3)

In [94]:
Lasso_grid_search.fit(X_train ,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END classifier=Lasso(), classifier__alpha=0.01;, score=0.171 total time=   3.3s
[CV 2/5] END classifier=Lasso(), classifier__alpha=0.01;, score=0.223 total time=   3.2s
[CV 3/5] END classifier=Lasso(), classifier__alpha=0.01;, score=0.226 total time=   3.0s
[CV 4/5] END classifier=Lasso(), classifier__alpha=0.01;, score=0.246 total time=   4.4s
[CV 5/5] END classifier=Lasso(), classifier__alpha=0.01;, score=0.225 total time=   3.2s
[CV 1/5] END classifier=Lasso(), classifier__alpha=0.1;, score=-0.001 total time=   2.8s
[CV 2/5] END classifier=Lasso(), classifier__alpha=0.1;, score=-0.001 total time=   2.8s
[CV 3/5] END classifier=Lasso(), classifier__alpha=0.1;, score=-0.002 total time=   2.9s
[CV 4/5] END classifier=Lasso(), classifier__alpha=0.1;, score=-0.001 total time=   2.8s
[CV 5/5] END classifier=Lasso(), classifier__alpha=0.1;, score=-0.004 total time=   3.3s
[CV 1/5] END classifier=Lasso(), classifier__alpha

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tranformer',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         MinMaxScaler(),
                                                                         ['price']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['country',
                                                                          'province',
                                                                          'region_1',
                                                                          'variety',
                                                                          'winery',
                                                                          'year']),
                                                                        ('descriptional',
                                                                         Pipeline(steps=[('squeez',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c...
                                                                                                                      'accentuated',
                                                                                                                      'accentuates',
                                                                                                                      'accentuating',
                                                                                                                      'acceptable',
                                                                                                                      'accessible',
                                                                                                                      'accompanied',
                                                                                                                      'accompanies',
                                                                                                                      'accompaniment', ...})),
                                                                                         ('toarray',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c5551af70>))]),
                                                                         ['pureTextdescription'])])),
                                       ('classifier', Lasso())]),
             param_grid=[{'classifier': [Lasso(alpha=0.01)],
                          'classifier__alpha': [0.01, 0.1, 1.0, 10, 100]}],
             scoring='r2', verbose=3)

In [95]:
Lasso_results_df = pd.DataFrame(Lasso_grid_search.cv_results_)
Lasso_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
params = Lasso_results_df.loc[Lasso_results_df.rank_test_score == 1, 'params'].values[0]
train_MSE = -Lasso_results_df.loc[Lasso_results_df.rank_test_score == 1, 'mean_test_score'].values[0]
test_MSE = mean_squared_error(y_test.values, LinearRegression_grid_search.predict(X_test))
results_df.loc[2] = (['Lasso', numerical_features + categorical_features, params, train_MSE, test_MSE])
results_df

,method,variables,parameters,train_MSE,test_MSE
0,KNN,"[price, country, province, region_1, variety, ...","{'classifier__n_neighbors': 12, 'classifier__w...",5.829481,5.812558
1,LinearRegression,"[price, country, province, region_1, variety, ...","{'classifier': LinearRegression(), 'classifier...",-0.277362,15.264439
2,Lasso,"[price, country, province, region_1, variety, ...","{'classifier': Lasso(alpha=0.01), 'classifier_...",-0.218290,15.264439


## Ridge

In [96]:
# scoring = 'neg_mean_squared_error'
scoring = 'r2'

In [97]:
param_grid = [{
         'classifier':[Ridge()],
         'classifier__alpha':np.arange(0.05,0.5,0.8)
     }]

In [98]:
Ridge_pipe = Pipeline([
    ('tranformer', x_tranformer),
    ('classifier', Ridge())
])

Ridge_grid_search = GridSearchCV(Ridge_pipe,param_grid, cv = 5, scoring = scoring, verbose = 3)

In [99]:
Ridge_grid_search.fit(X_train ,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END classifier=Ridge(), classifier__alpha=0.05;, score=0.368 total time=   5.5s
[CV 2/5] END classifier=Ridge(), classifier__alpha=0.05;, score=0.375 total time=   5.5s
[CV 3/5] END classifier=Ridge(), classifier__alpha=0.05;, score=0.418 total time=   6.3s
[CV 4/5] END classifier=Ridge(), classifier__alpha=0.05;, score=0.440 total time=   6.5s
[CV 5/5] END classifier=Ridge(), classifier__alpha=0.05;, score=0.401 total time=   5.6s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tranformer',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         MinMaxScaler(),
                                                                         ['price']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['country',
                                                                          'province',
                                                                          'region_1',
                                                                          'variety',
                                                                          'winery',
                                                                          'year']),
                                                                        ('descriptional',
                                                                         Pipeline(steps=[('squeez',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c...
                                                                                                                      'accentuated',
                                                                                                                      'accentuates',
                                                                                                                      'accentuating',
                                                                                                                      'acceptable',
                                                                                                                      'accessible',
                                                                                                                      'accompanied',
                                                                                                                      'accompanies',
                                                                                                                      'accompaniment', ...})),
                                                                                         ('toarray',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c5551af70>))]),
                                                                         ['pureTextdescription'])])),
                                       ('classifier', Ridge())]),
             param_grid=[{'classifier': [Ridge(alpha=0.05)],
                          'classifier__alpha': array([0.05])}],
             scoring='r2', verbose=3)

In [100]:
Ridge_results_df = pd.DataFrame(Ridge_grid_search.cv_results_)
Ridge_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
params = Ridge_results_df.loc[Ridge_results_df.rank_test_score == 1, 'params'].values[0]
train_MSE = -Ridge_results_df.loc[Ridge_results_df.rank_test_score == 1, 'mean_test_score'].values[0]
test_MSE = mean_squared_error(y_test.values, Ridge_grid_search.predict(X_test))
results_df.loc[3] = (['Ridge', numerical_features + categorical_features, params, train_MSE, test_MSE])
results_df

,method,variables,parameters,train_MSE,test_MSE
0,KNN,"[price, country, province, region_1, variety, ...","{'classifier__n_neighbors': 12, 'classifier__w...",5.829481,5.812558
1,LinearRegression,"[price, country, province, region_1, variety, ...","{'classifier': LinearRegression(), 'classifier...",-0.277362,15.264439
2,Lasso,"[price, country, province, region_1, variety, ...","{'classifier': Lasso(alpha=0.01), 'classifier_...",-0.218290,15.264439
3,Ridge,"[price, country, province, region_1, variety, ...","{'classifier': Ridge(alpha=0.05), 'classifier_...",-0.400409,5.324870


## DecisionTreeRegressor

In [101]:
# scoring = 'neg_mean_squared_error'
scoring = 'r2'

In [102]:
 param_grid = [{
        'classifier': [DecisionTreeRegressor()],
        'classifier__max_depth': [7,10,12,13],
        'classifier__min_samples_leaf':[15,25,35],        
    }]

In [103]:
DecisionTreeRegressor_pipe = Pipeline([
    ('tranformer', x_tranformer),
    ('classifier', DecisionTreeRegressor())
])

DecisionTreeRegressor_grid_search = GridSearchCV(DecisionTreeRegressor_pipe,param_grid, cv = 5, scoring = scoring, verbose = 3)

In [104]:
DecisionTreeRegressor_grid_search.fit(X_train ,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END classifier=DecisionTreeRegressor(), classifier__max_depth=7, classifier__min_samples_leaf=15;, score=0.374 total time=  10.2s
[CV 2/5] END classifier=DecisionTreeRegressor(), classifier__max_depth=7, classifier__min_samples_leaf=15;, score=0.375 total time=   7.7s
[CV 3/5] END classifier=DecisionTreeRegressor(), classifier__max_depth=7, classifier__min_samples_leaf=15;, score=0.341 total time=   7.8s
[CV 4/5] END classifier=DecisionTreeRegressor(), classifier__max_depth=7, classifier__min_samples_leaf=15;, score=0.415 total time=   7.3s
[CV 5/5] END classifier=DecisionTreeRegressor(), classifier__max_depth=7, classifier__min_samples_leaf=15;, score=0.364 total time=   7.7s
[CV 1/5] END classifier=DecisionTreeRegressor(), classifier__max_depth=7, classifier__min_samples_leaf=25;, score=0.374 total time=   8.5s
[CV 2/5] END classifier=DecisionTreeRegressor(), classifier__max_depth=7, classifier__min_samples_leaf=25

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tranformer',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         MinMaxScaler(),
                                                                         ['price']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['country',
                                                                          'province',
                                                                          'region_1',
                                                                          'variety',
                                                                          'winery',
                                                                          'year']),
                                                                        ('descriptional',
                                                                         Pipeline(steps=[('squeez',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c...
                                                                                                                      'accompaniment', ...})),
                                                                                         ('toarray',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c5551af70>))]),
                                                                         ['pureTextdescription'])])),
                                       ('classifier',
                                        DecisionTreeRegressor())]),
             param_grid=[{'classifier': [DecisionTreeRegressor(max_depth=13,
                                                               min_samples_leaf=25)],
                          'classifier__max_depth': [7, 10, 12, 13],
                          'classifier__min_samples_leaf': [15, 25, 35]}],
             scoring='r2', verbose=3)

In [105]:
DecisionTreeRegressor_results_df = pd.DataFrame(DecisionTreeRegressor_grid_search.cv_results_)
DecisionTreeRegressor_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
params = DecisionTreeRegressor_results_df.loc[DecisionTreeRegressor_results_df.rank_test_score == 1, 'params'].values[0]
train_MSE = -DecisionTreeRegressor_results_df.loc[DecisionTreeRegressor_results_df.rank_test_score == 1, 'mean_test_score'].values[0]
test_MSE = mean_squared_error(y_test.values, DecisionTreeRegressor_grid_search.predict(X_test))
results_df.loc[4] = (['DecisionTreeRegressor', numerical_features + categorical_features, params, train_MSE, test_MSE])
results_df

,method,variables,parameters,train_MSE,test_MSE
0,KNN,"[price, country, province, region_1, variety, ...","{'classifier__n_neighbors': 12, 'classifier__w...",5.829481,5.812558
1,LinearRegression,"[price, country, province, region_1, variety, ...","{'classifier': LinearRegression(), 'classifier...",-0.277362,15.264439
2,Lasso,"[price, country, province, region_1, variety, ...","{'classifier': Lasso(alpha=0.01), 'classifier_...",-0.218290,15.264439
3,Ridge,"[price, country, province, region_1, variety, ...","{'classifier': Ridge(alpha=0.05), 'classifier_...",-0.400409,5.324870
4,DecisionTreeRegressor,"[price, country, province, region_1, variety, ...",{'classifier': DecisionTreeRegressor(max_depth...,-0.379039,5.641304


## RandomForestRegressor

In [106]:
# scoring = 'neg_mean_squared_error'
scoring = 'r2'

In [107]:
 param_grid = [{
        'classifier':[RandomForestRegressor()],
        'classifier__max_depth': [9,11],
        'classifier__min_samples_leaf':[15,30]
    }]

In [108]:
RandomForestRegressor_pipe = Pipeline([
    ('tranformer', x_tranformer),
    ('classifier', Ridge())
])

RandomForestRegressor_grid_search = GridSearchCV(RandomForestRegressor_pipe,param_grid, cv = 5, scoring = scoring, verbose = 3)

In [109]:
RandomForestRegressor_grid_search.fit(X_train ,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END classifier=RandomForestRegressor(), classifier__max_depth=9, classifier__min_samples_leaf=15;, score=0.417 total time= 5.8min
[CV 2/5] END classifier=RandomForestRegressor(), classifier__max_depth=9, classifier__min_samples_leaf=15;, score=0.421 total time= 7.2min
[CV 3/5] END classifier=RandomForestRegressor(), classifier__max_depth=9, classifier__min_samples_leaf=15;, score=0.386 total time= 7.6min
[CV 4/5] END classifier=RandomForestRegressor(), classifier__max_depth=9, classifier__min_samples_leaf=15;, score=0.458 total time=180.7min
[CV 5/5] END classifier=RandomForestRegressor(), classifier__max_depth=9, classifier__min_samples_leaf=15;, score=0.421 total time= 5.1min
[CV 1/5] END classifier=RandomForestRegressor(), classifier__max_depth=9, classifier__min_samples_leaf=30;, score=0.401 total time= 6.2min
[CV 2/5] END classifier=RandomForestRegressor(), classifier__max_depth=9, classifier__min_samples_leaf=30

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tranformer',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         MinMaxScaler(),
                                                                         ['price']),
                                                                        ('categorical',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['country',
                                                                          'province',
                                                                          'region_1',
                                                                          'variety',
                                                                          'winery',
                                                                          'year']),
                                                                        ('descriptional',
                                                                         Pipeline(steps=[('squeez',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c...
                                                                                                                      'accompanied',
                                                                                                                      'accompanies',
                                                                                                                      'accompaniment', ...})),
                                                                                         ('toarray',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x7f8c5551af70>))]),
                                                                         ['pureTextdescription'])])),
                                       ('classifier', Ridge())]),
             param_grid=[{'classifier': [RandomForestRegressor(max_depth=11,
                                                               min_samples_leaf=15)],
                          'classifier__max_depth': [9, 11],
                          'classifier__min_samples_leaf': [15, 30]}],
             scoring='r2', verbose=3)

In [110]:
RandomForestRegressor_results_df = pd.DataFrame(RandomForestRegressor_grid_search.cv_results_)
RandomForestRegressor_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
params = RandomForestRegressor_results_df.loc[RandomForestRegressor_results_df.rank_test_score == 1, 'params'].values[0]
train_MSE = -RandomForestRegressor_results_df.loc[RandomForestRegressor_results_df.rank_test_score == 1, 'mean_test_score'].values[0]
test_MSE = mean_squared_error(y_test.values, RandomForestRegressor_grid_search.predict(X_test))
results_df.loc[5] = (['RandomForestRegressor', numerical_features + categorical_features, params, train_MSE, test_MSE])
results_df

,method,variables,parameters,train_MSE,test_MSE
0,KNN,"[price, country, province, region_1, variety, ...","{'classifier__n_neighbors': 12, 'classifier__w...",5.829481,5.812558
1,LinearRegression,"[price, country, province, region_1, variety, ...","{'classifier': LinearRegression(), 'classifier...",-0.277362,15.264439
2,Lasso,"[price, country, province, region_1, variety, ...","{'classifier': Lasso(alpha=0.01), 'classifier_...",-0.218290,15.264439
3,Ridge,"[price, country, province, region_1, variety, ...","{'classifier': Ridge(alpha=0.05), 'classifier_...",-0.400409,5.324870
4,DecisionTreeRegressor,"[price, country, province, region_1, variety, ...",{'classifier': DecisionTreeRegressor(max_depth...,-0.379039,5.641304
5,RandomForestRegressor,"[price, country, province, region_1, variety, ...",{'classifier': RandomForestRegressor(max_depth...,-0.426322,5.272509


##### Create pipline With the manipulate data transform

In [54]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Define categorical and numerical features
categorical_features =  ['country','designation','province','region_1','region_2','taster_name','taster_twitter_handle','title','variety','winery','year']
# categorical_features =  ['country','province','taster_name','title','variety','winery']

numerical_features = ['price']
description_features = ['pureTextdescription']

# Create categorical and numerical transformers
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


description_transformer = Pipeline(steps=[
    ("squeez", FunctionTransformer(lambda x: x.squeeze())),
    #("vect", CountVectorizer(vocabulary = clean_vocab, tokenizer=nltk.word_tokenize)),
    ("tfidf", TfidfVectorizer(vocabulary=clean_vocab)),
    ("toarray", FunctionTransformer(lambda x: x.toarray())),
])

# Combine transformers using ColumnTransformer
wine_reviews_preprocessor = ColumnTransformer(
    transformers=[
        ('desc',description_transformer,description_features),
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
        
], remainder='drop')

In [55]:
wine_reviews_preprocessor

ColumnTransformer(transformers=[('desc',
                                 Pipeline(steps=[('squeez',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fde544a21f0>)),
                                                 ('tfidf',
                                                  TfidfVectorizer(vocabulary={'abound',
                                                                              'acacia',
                                                                              'accent',
                                                                              'accented',
                                                                              'accents',
                                                                              'accessible',
                                                                              'acid',
                                                                              'acidic',
                                                                              'acidity',
                                                                              'acids',
                                                                              'across',
                                                                              'add',
                                                                              'added',
                                                                              'adding',
                                                                              'addition',
                                                                              'additional',
                                                                              'adds',
                                                                              'aftertaste',
                                                                              'age',
                                                                              'aged',
                                                                              '...
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['price']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['country', 'designation', 'province',
                                  'region_1', 'region_2', 'taster_name',
                                  'taster_twitter_handle', 'title', 'variety',
                                  'winery'])])

In [67]:
categorical_transformer.fit_transform(X_train[['country']])

Column Name :  country
Column Contents :  <class 'str'>
Column Name :  designation
Column Contents :  <class 'str'>
Column Name :  province
Column Contents :  <class 'str'>
Column Name :  region_1
Column Contents :  <class 'str'>
Column Name :  region_2
Column Contents :  <class 'str'>
Column Name :  taster_name
Column Contents :  <class 'str'>
Column Name :  taster_twitter_handle
Column Contents :  <class 'str'>
Column Name :  title
Column Contents :  <class 'str'>
Column Name :  variety
Column Contents :  <class 'str'>
Column Name :  winery
Column Contents :  <class 'str'>


array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

##### Fit Tramsform data before use different models.

In [68]:
from sklearn import set_config

In [69]:
# set_config(transform_output='pandas')

In [70]:
res=wine_reviews_preprocessor.fit_transform(X_train)
# .agg(['mean','std'])
res[np.where(res[:,0] == 1)]

array([], shape=(0, 19720), dtype=float64)

## Run Models

In this area I will run different models to see the result and compare the result of each model.

The second step will be to find the best ridge on this models.

The models We Will Check:
* LogiticRegression.
* SVM rbf with Kernal.
* Random Forest.
* KNN.
* Lasso.
* Ridge.

##### Logistic Regression

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC

# Create a full pipeline with a classifier
full_pipeline = Pipeline(steps=[
    ('preprocessor', wine_reviews_preprocessor),
    ('classifier', LogisticRegression(solver='liblinear'))
])

# Fit the pipeline to the training data
full_pipeline.fit(X_train, y_train)

# Evaluate the pipeline on the test data
accuracy = full_pipeline.score(X_test, y_test)
print(f"Test accuracy: {accuracy:.3f}")

Test accuracy: 0.261


In [72]:
param_grid = [
    { 
        'classifier': [LinearRegression()],
        'classifier__fit_intercept': [True, False]
    },
    {
        'classifier':[Lasso()],
        'classifier__alpha':[0.01,0.1, 1.0, 10, 100]
    },
    
     {
         'classifier':[Ridge()],
         'classifier__alpha':np.arange(0.05,0.5,0.8)
     },
    {
        'classifier': [DecisionTreeRegressor()],
        'classifier__max_depth': [7,10,12,13],
        'classifier__min_samples_leaf':[15,25,35],        
    },
    

    {
        'classifier':[RandomForestRegressor()],
        'classifier__max_depth': [9,11],
        'classifier__min_samples_leaf':[15,30]
    },
]

In [73]:
grid_search = GridSearchCV(full_pipeline, param_grid,  scoring='r2', cv=2, n_jobs=-1, verbose=2)


In [ ]:
import pdb;pdb.set_trace()
grid_search.fit(X_train, y_train)


--Return--
None
> /var/folders/y9/5q7dx2j91_x2pwj_s_kllqc80000gp/T/ipykernel_11817/3379891941.py(1)<module>()
----> 1 import pdb;pdb.set_trace()
      2 grid_search.fit(X_train, y_train)

[CV] END ..........classifier=Lasso(), classifier__alpha=0.1; total time=  22.9s
[CV] END classifier=DecisionTreeRegressor(), classifier__max_depth=7, classifier__min_samples_leaf=15; total time=  11.7s
[CV] END classifier=DecisionTreeRegressor(), classifier__max_depth=10, classifier__min_samples_leaf=15; total time=  12.8s
[CV] END classifier=DecisionTreeRegressor(), classifier__max_depth=12, classifier__min_samples_leaf=25; total time=  16.6s
[CV] END classifier=DecisionTreeRegressor(), classifier__max_depth=13, classifier__min_samples_leaf=35; total time=  15.8s
[CV] END classifier=RandomForestRegressor(), classifier__max_depth=11, classifier__min_samples_leaf=30; total time= 7.9min
[CV] END ...........classifier=Lasso(), classifier__alpha=10; total time=  18.9s
[CV] END ..........classifier=Lasso(

In [ ]:
grid_search.best_params_

In [ ]:
R_2 = grid_search.score(X_test, y_test)
print(f"Test R^2: {R_2:.3f}")

In [ ]:
results_df = pd.DataFrame(grid_search.cv_results_)
results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']].sort_values(by=['rank_test_score'])


In [ ]:
# Clean parameter name and extract classifier name
# results_df['classifier_name'] = results_df['params']['classifier']
# row['classifier']
# Print the cleaned classifier names
# print(results_df['classifier_name'])
results_df['classifier_name']=[row['classifier'] for row in results_df['params']]
results_df[['classifier_name','params', 'mean_test_score', 'std_test_score', 'rank_test_score']].sort_values(by=['rank_test_score'])

In [ ]:
# pip install tqdm

In [ ]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
        context_class = tf.expand_dims(
            tf.constant([context_word], dtype="int64"), 1)
        negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
            true_classes=context_class,
            num_true=1,
            num_sampled=num_ns,
            unique=True,
            range_max=vocab_size,
            seed=SEED,
            name="negative_sampling")

    # Build context and label vectors (for one target word)
    negative_sampling_candidates = tf.expand_dims(
        negative_sampling_candidates, 1)
    context = tf.concat([context_class, negative_sampling_candidates], 0)
    label = tf.constant([1] + [0]*num_ns, dtype="int64")

    # Append each element from the training example to global lists.
    targets.append(target_word)
    contexts.append(context)
    labels.append(label)

    return targets, contexts, labels